In [12]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
inputfolder = 'output/Enhanced/'
outputfolder = 'output/Jaw/'

In [13]:
def thesholding(res2):
    res2 = cv2.cvtColor(res2,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(res2,(5,5),0)
    ret3,th3 = cv2.threshold(blur,100,255,cv2.THRESH_BINARY)
    return th3
    #cv2.imwrite("output/thresholding.jpg",th3)

In [14]:
def patching(th3):
    sizeX = th3.shape[1] # columns
    sizeY = th3.shape[0] # rows
    nRows = 150
    mCols = 150
    x_increment = int(sizeY/nRows)
    y_increment = int(sizeX/mCols)
    
    for i in range(0,nRows):
        for j in range(0, mCols):
            roi = th3[int(i*x_increment):int(i*x_increment + x_increment) ,int(j*y_increment):int(j*y_increment + y_increment)]
            count = cv2.countNonZero(roi)
            if 100*count/roi.size >= 60:
                th3[int(i*x_increment):int(i*x_increment + x_increment) ,int(j*y_increment):int(j*y_increment + y_increment)] = 255
            else:
                th3[int(i*x_increment):int(i*x_increment + x_increment) ,int(j*y_increment):int(j*y_increment + y_increment)] = 0
    return th3
    #cv2.imwrite("output/patching.jpg",th3)

In [15]:
def plotting(th3):
    
    cols = th3.shape[1] 
    rows = th3.shape[0]
    
    points = list()
    left= list()
    right = list()
    
    #lower_jaw = th3.copy()
    #jaw lowest point
    
    i = rows - 1

    while i>0:
        if th3[i][int(cols/2) -1] == 255:
            i = i -1
        else:
            points.append([i,int(cols/2) -1 ])
            break

    midx = points[0][0]
    midy = points[0][1]

    #midpoint
    mid1 = [midx,int(midy + ((cols - midy) /2))]
    mid2 = [midx,int(midy /2)] 

    #lower_jaw[mid1[0],mid1[1]] = 150
    #lower_jaw[mid2[0],mid2[1]] = 150
    
    r = midx
    right_point = [0,0]
    left_point = [0,0]
    
    for i in range(r-1,1,-1):
        midr = mid1[1]
        midl = mid2[1]
    
    #while pixeled[i][midr] == 255 and midr > 0:
     #   midr = midr - 3
     
    #while pixeled[i][midl] == 255 and midl > 0:
     #   midl = midl -3
    
        if th3[i][midr] == 255 and midr > 0:
            midr = midr - 3
            continue
     
        if th3[i][midl] == 255 and midl > 0:
            midl = midl + 3
            continue
        
    #lower_jaw[i][midr] = 150
    #lower_jaw[i][midl] = 150
        fr = 0
        fl = 0
    
        for k in range(midr,int(cols/2),-1):
            if th3[i][k] == 255:
            #print(k,pixeled.shape[1]/2,pixeled[i][k])
            #lower_jaw[i][k] = 200
                right_point = (i,k)
                right.append([i,k])
                fr = 1
                break
            
        for j in range(midl,int(cols/2)):
            if j<0:
                break
            
            if th3[i][j] == 255:
            #print(midl)
            #lower_jaw[i][j] = 200
                left_point = (i,j)
            #print(left_point)
                left.append([i,j])
                fl = 1
                break
            
        if fr == 1:
            for m in range(midr,cols-1):
                if th3[i][m] == 255:
                    break
            mid1 =[right_point[0],int(right_point[1] + ((m- right_point[1]) /2))]
        
        if fl == 1:
            for m in range(midl,0,-1):
                if th3[i][m] == 255:
                    break
            mid2 =[left_point[0],int((m+left_point[1])/2)]
        
    #p = left + right
    return left,right    
#cv2.imwrite('output/lower_jaw.jpg',lower_jaw) 

In [16]:
def correct_points(left,right,th3):
    for i in range(1,len(left)-2):
        if (abs(abs(left[i][1] - left[i-1][1]) - abs(left[i][1] - left[i+1][1]))) > 4:
            left[i][1] = int((left[i-1][1] + left[i+1][1]) / 2)
        
    for i in range(1,len(right)-2):
        if (abs(abs(right[i][1] - right[i-1][1]) - abs(right[i][1] - right[i+1][1]))) > 4:
            right[i][1] = int((right[i-1][1] + right[i+1][1]) / 2)
        
    corr = np.zeros((th3.shape[0],th3.shape[1],3))
    for i,j in left:
        corr[i][j] = [255,255,255]
    for i,j in right:
        corr[i][j] = [255,255,255]
    path = outputfolder +'/' + 'final_points.jpg'
    cv2.imwrite(path,corr)
    points = left + right
    return points
    #corrected = np.hstack((black,corr))
    #cv2.imwrite('output/corrected_points.jpg',corrected)

In [17]:
def remove_noise():
    path = outputfolder +'/' + 'final_points.jpg'
    po = cv2.imread(path,0)
    sizeX = po.shape[1] # columns
    sizeY = po.shape[0] # rows
    rows = sizeY
    cols = sizeX
    nRows = 100
    mCols = 10
    x_increment = int(sizeY/nRows)
    y_increment = int(sizeX/mCols)

    for i in range(0,nRows):
        roi = po[int(i*x_increment):int(i*x_increment + x_increment) ,:]
        count = cv2.countNonZero(roi)
        if count < 200:
            po[int(i*x_increment):int(i*x_increment + x_increment) ,:] = 0

    for j in range(0,nRows):
        roi = po[:, int(j*y_increment):int(j*y_increment + y_increment)]
        count = cv2.countNonZero(roi)
        if count < 100:
            po[:, int(j*y_increment):int(j*y_increment + y_increment)] = 0
    path = outputfolder +'/' + 'removed_points.jpg'
    cv2.imwrite(path,po)
    
    point = list()
    path = outputfolder +'/' + 'removed_points.jpg'
    ir = cv2.imread(path,0)
    for i in range(0,rows):
        for j in range(0,cols):
            if ir[i][j] == 255:
                point.append([i,j])
    return point

In [18]:
def interchange_x_y(point):
    for i in range(0,len(point)):
        point[i][0],point[i][1] = point[i][1],point[i][0]
    return point

In [19]:
def fit_ellipse(point,img):
    ellipse = cv2.fitEllipse(np.float32(point))
    black = np.zeros((img.shape[0],img.shape[1],1))
    im = cv2.ellipse(black,ellipse,255,2)
    path = outputfolder +'/' + 'temp.jpg'
    cv2.imwrite(path,im)

In [20]:
def remove_point_noise(Inputfolder,outputfolder,filename,img):
    path = outputfolder +'/' + 'temp.jpg'
    image1 = cv2.imread(path,0)
    final = img.copy()
    cols = image1.shape[1] 
    rows = image1.shape[0]
    
    for i in range(0,cols):
        for j in range(0,rows):
            if image1[j][i] == 255:
                xMax = i
                break

    for i in range(cols-1,0,-1):
        for j in range(0,rows):
            if image1[j][i] == 255:
                xMin = i 
                break

    for i in range(0,rows):
        for j in range(0,cols):
            if image1[i][j] == 255:
                yMax = i
                break

    for i in range(rows-1,0,-1):
        for j in range(0,cols):
            if image1[i][j] == 255:
                yMin = i 
                break
                
    center = (int((xMax + xMin)/2),int((yMax+yMin)/2))
    k=0.9
    height = int(((yMax-yMin) * k)/2)
    width = int((height*2))
    axis = (width,height)
    #fitted_ellipse = cv2.ellipse(im1,center,axis,0,0,360,(255,255,255),3)
    #cv2.imwrite("output/corrected_ellipse.jpg",fitted_ellipse)
    
    mask = np.ones(image1.shape[:2], dtype="uint8") * 0
    fitted_ellipse = cv2.ellipse(mask,center,axis,0,0,360,(255,255,255),-1)
    path = outputfolder +'/' + 'mask.jpg'
    cv2.imwrite(path,fitted_ellipse)
    roi = cv2.bitwise_and(final,final,mask = mask)
    output = outputfolder + '/' + filename +'.jpg'
    mask = cv2.bitwise_not(mask)
    background = np.full(roi.shape, 255, dtype=np.uint8)
    bk = cv2.bitwise_or(roi, background, mask=mask)
    bk = cv2.bitwise_or(roi,bk)
    cv2.imwrite(output,bk)

In [21]:
def load_images_from_folder(inputfolder,outputfolder):
    for filename in os.listdir(inputfolder):
        splitted_data = filename.split('_')
        Image_id = splitted_data[1].split('.')
        Image_id = Image_id[0]
        img = cv2.imread(os.path.join(inputfolder,filename))
        th3 = thesholding(img)
        th3 = patching(th3)
        left,right = plotting(th3)
        points = correct_points(left,right,th3)
        point = remove_noise()
        point = interchange_x_y(point)
        fit_ellipse(point,img)
        remove_point_noise(inputfolder,outputfolder,Image_id,img)
        print(Image_id + '  ..................done')

In [22]:
load_images_from_folder(inputfolder,outputfolder)

enhance1  ..................done
enhance10  ..................done
enhance11  ..................done
enhance12  ..................done
enhance13  ..................done
enhance14  ..................done
enhance15  ..................done
enhance2  ..................done
enhance3  ..................done
enhance4  ..................done
enhance5  ..................done
enhance6  ..................done
enhance7  ..................done
enhance8  ..................done
enhance9  ..................done
